In [1]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/Computational-Personality-Recognition/essays.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_essays))

2469


In [2]:
data_essays.drop(data_essays.head(1).index, inplace=True)

In [3]:
NRCDict = {}
with open("NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt",'r') as f:
    for line in f:
        text = line.split()
        if text[2] == '1':
            if text[0] in NRCDict:
                NRCDict[text[0]] += "#" + text[1]
            else:
                NRCDict[text[0]] = text[1]

In [4]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from empath import Empath
import numpy as np
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.preprocessing import normalize
import nltk

accuracy_DT = 0.0
rmse_DT = [0.0, 0.0, 0.0, 0.0, 0.0]
h_loss_DT = 0.0

accuracy_RF = 0.0
rmse_RF = [0.0, 0.0, 0.0, 0.0, 0.0]
h_loss_RF = 0.0

for i in range(10):
    rand_essays = []
    temp_data = data_essays.copy()
    rand_essays = random.sample(range(1,len(temp_data)-1),250)
    test = temp_data.iloc[rand_essays].copy()
    train = temp_data.drop(rand_essays)
    train.drop(train.columns[[1]], axis=1, inplace=True)
    test.drop(test.columns[[1]], axis=1, inplace=True)
    
    corpus = []
    truth = []
    for row in train.iterrows():
        Y = [0,0,0,0,0]
        corpus.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth.append(Y)
    truth = np.array(truth)
    feat = []
    for row in corpus:
        tokens = nltk.word_tokenize(row)
        feat_row = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        for word in tokens:
            if word in NRCDict:
                category = NRCDict[word].split("#")
                if "anger" in category: feat_row[0] += 1
                if "anticipation" in category: feat_row[1] += 1
                if "disgust" in category: feat_row[2] += 1
                if "fear" in category: feat_row[3] += 1
                if "joy" in category: feat_row[4]+= 1
                if "negative" in category: feat_row[5] += 1
                if "positive" in category: feat_row[6] += 1
                if "sadness" in category: feat_row[7] += 1
                if "surprise" in category: feat_row[8] += 1
                if "trust" in category: feat_row[9] += 1
            else: continue
        feat.append(feat_row)
    feat = np.array(feat)
    feat_norm = normalize(feat, axis=1, norm='l1')
    clf_DT = tree.DecisionTreeClassifier()
    clf_DT = clf_DT.fit(feat_norm, truth)
    
    clf_RF = RandomForestClassifier()
    clf_RF = clf_RF.fit(feat_norm, truth)
    
    test_data = []
    truth_test = []
    for row in test.iterrows():
        Y = [0,0,0,0,0]
        test_data.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth_test.append(Y)
    truth_test = np.array(truth_test)
    test_feat = []
    for row in test_data:
        tokens = nltk.word_tokenize(row)
        feat_row = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        for word in tokens:
            if word in NRCDict:
                category = NRCDict[word].split("#")
                if "anger" in category: feat_row[0] += 1
                if "anticipation" in category: feat_row[1] += 1
                if "disgust" in category: feat_row[2] += 1
                if "fear" in category: feat_row[3] += 1
                if "joy" in category: feat_row[4] += 1
                if "negative" in category: feat_row[5] += 1
                if "positive" in category: feat_row[6] += 1
                if "sadness" in category: feat_row[7] += 1
                if "surprise" in category: feat_row[8] += 1
                if "trust" in category: feat_row[9] += 1
            else: continue
        test_feat.append(feat_row)
    test_feat = np.array(test_feat)
    test_feat_norm = normalize(test_feat, axis=1, norm='l1')
    
    print("----------------- Iteration : " + str(i) + "-----------------")
    print("Results for DT")
    output_DT = clf_DT.predict(test_feat_norm)
    acc = accuracy_score(truth_test, output_DT, normalize=True)
    loss = hamming_loss(truth_test, output_DT)
    err = [sqrt(mean_squared_error(truth_test[:,i], output_DT[:,i])) for i in range(len(truth_test[0,:]))] 
    accuracy_DT += acc
    h_loss_DT += loss
    print("Accuracy : " + str(acc))
    for per_type in range(len(rmse_DT)):
        rmse_DT[per_type] += err[per_type]
    print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
    print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
    print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
    print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
    print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))
    print("Hamming loss : " + str(loss))
    print("")
    print("Results for RF")
    output_RF = clf_RF.predict(test_feat_norm)
    acc = accuracy_score(truth_test, output_RF, normalize=True)
    loss = hamming_loss(truth_test, output_RF)
    err = [sqrt(mean_squared_error(truth_test[:,i], output_RF[:,i])) for i in range(len(truth_test[0,:]))] 
    accuracy_RF += acc
    h_loss_RF += loss
    print("Accuracy : " + str(acc))
    for per_type in range(len(rmse_RF)):
        rmse_RF[per_type] += err[per_type]
    print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
    print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
    print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
    print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
    print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))
    print("Hamming loss : " + str(loss))
    del test
    del train
    del temp_data
print("--------------------------------")
print("Results for DT:")
print("Average accuracy is :" + str(accuracy_DT/10))
print("Average RMSE - AGR is " + ": " + str(rmse_DT[0]/10))
print("Average RMSE - CON is " + ": " + str(rmse_DT[1]/10))
print("Average RMSE - EXT is " + ": " + str(rmse_DT[2]/10))
print("Average RMSE - NEU is " + ": " + str(rmse_DT[3]/10))
print("Average RMSE - OPN is " + ": " + str(rmse_DT[4]/10))
print("Average Hamming loss is :" + str(h_loss_DT/10))
print("")
print("Results for RF:")
print("Average accuracy is :" + str(accuracy_RF/10))
print("Average RMSE - AGR is " + ": " + str(rmse_RF[0]/10))
print("Average RMSE - CON is " + ": " + str(rmse_RF[1]/10))
print("Average RMSE - EXT is " + ": " + str(rmse_RF[2]/10))
print("Average RMSE - NEU is " + ": " + str(rmse_RF[3]/10))
print("Average RMSE - OPN is " + ": " + str(rmse_RF[4]/10))
print("Average Hamming loss is :" + str(h_loss_RF/10))

----------------- Iteration : 0-----------------
Results for DT
Accuracy : 0.9
RMSE - AGR for interation 0: 0.21908902300206645
RMSE - CON for interation 0: 0.2
RMSE - EXT for interation 0: 0.21908902300206645
RMSE - NEU for interation 0: 0.20976176963403032
RMSE - OPN for interation 0: 0.2449489742783178
Hamming loss : 0.048

Results for RF
Accuracy : 0.884
RMSE - AGR for interation 0: 0.22803508501982758
RMSE - CON for interation 0: 0.23664319132398465
RMSE - EXT for interation 0: 0.2
RMSE - NEU for interation 0: 0.23664319132398465
RMSE - OPN for interation 0: 0.2
Hamming loss : 0.0488
----------------- Iteration : 1-----------------
Results for DT
Accuracy : 0.912
RMSE - AGR for interation 1: 0.20976176963403032
RMSE - CON for interation 1: 0.23664319132398465
RMSE - EXT for interation 1: 0.2449489742783178
RMSE - NEU for interation 1: 0.17888543819998318
RMSE - OPN for interation 1: 0.18973665961010275
Hamming loss : 0.0456

Results for RF
Accuracy : 0.848
RMSE - AGR for interatio

In [5]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/Computational-Personality-Recognition/fb.json") as f:
    data_fb = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_fb))

9917


In [6]:
data_fb.drop(data_fb.head(1).index, inplace=True)

In [7]:
test_fb = data_fb.copy()
test_fb.drop(test_fb.columns[[1,7,8,9,10,11]], axis=1, inplace=True)

corpus_fb = []
truth_fb = []
for row in test_fb.iterrows():
    Y = [0,0,0,0,0]
    corpus_fb.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth_fb.append(Y)
truth_fb = np.array(truth_fb)
feat_fb = []
for row in corpus_fb:
    tokens = nltk.word_tokenize(row)
    feat_row = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
    for word in tokens:
        if word in NRCDict:
            category = NRCDict[word].split("#")
            if "anger" in category: feat_row[0] += 1
            if "anticipation" in category: feat_row[1] += 1
            if "disgust" in category: feat_row[2] += 1
            if "fear" in category: feat_row[3] += 1
            if "joy" in category: feat_row[4]+= 1
            if "negative" in category: feat_row[5] += 1
            if "positive" in category: feat_row[6] += 1
            if "sadness" in category: feat_row[7] += 1
            if "surprise" in category: feat_row[8] += 1
            if "trust" in category: feat_row[9] += 1
        else: continue
    feat_fb.append(feat_row)
feat_fb = np.array(feat_fb)
feat_fb_norm = normalize(feat_fb, axis=1, norm='l1')

print("Results for DT")
output_fb_DT = clf_DT.predict(feat_fb_norm)

print("Accuracy : " + str(accuracy_score(truth_fb, output_fb_DT, normalize=True)))
print("Hamming Loss : " + str(hamming_loss(truth_fb, output_fb_DT)))
err = [sqrt(mean_squared_error(truth_fb[:,i], output_fb_DT[:,i])) for i in range(len(truth_fb[0,:]))]
print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))

print("Results for RF")
output_fb_RF = clf_RF.predict(feat_fb_norm)

print("Accuracy : " + str(accuracy_score(truth_fb, output_fb_RF, normalize=True)))
print("Hamming Loss : " + str(hamming_loss(truth_fb, output_fb_RF)))
err = [sqrt(mean_squared_error(truth_fb[:,i], output_fb_RF[:,i])) for i in range(len(truth_fb[0,:]))]
print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))

Results for DT
Accuracy : 0.0493142396127
Hamming Loss : 0.5076442113755547
RMSE - AGR for interation 9: 0.6971242722123682
RMSE - CON for interation 9: 0.721017948325269
RMSE - EXT for interation 9: 0.7438737286957019
RMSE - NEU for interation 9: 0.6566751609762338
RMSE - OPN for interation 9: 0.7401361583358563
Results for RF
Accuracy : 0.0294473578056
Hamming Loss : 0.5248487293263413
RMSE - AGR for interation 9: 0.7003715950246598
RMSE - CON for interation 9: 0.71680963641739
RMSE - EXT for interation 9: 0.7446189915098487
RMSE - NEU for interation 9: 0.6416843255831267
RMSE - OPN for interation 9: 0.8085115982082589
